This is a project to build a model that diagnoses eye conditions based on images of the anterior of the eye.
For this version we have a dataset with five different conditions, Cataract, Conjunctivitis, Eyelid, Normal and Uveitis.
For this model we are using a Sequential Convolutional Neural Network for this project.

In [1]:
# Step 1: Import Necessary Libraries
# In order to build the CNN, I need TensorFlow for the model, NumPy for data handling, cv2 for image processing, os for file manipulation,
# and Matplotlib for visualization.

import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# Output version of TensorFlow being used
print("TensorFlow version:", tf.__version__)

TensorFlow version: 2.16.2


In [2]:
# Step 2: Import Images from a data set where the different eye conditions have pictures in different directories.
# As we import the images we need to assign a class that uses the director name

# Set path for the director of the data set to be used
DATA_DIR = '/Users/aresbandebo/PycharmProjects/Eye_CNN_Testing/Dataset_Eye_Diseases_Classification/'

# Set the desired image dimensions for resizing.
IMG_HEIGHT = 224
IMG_WIDTH = 224

In [3]:
import re

def create_classes_from_dirs(target_path):
    """
    Scans a target path and dynamically creates Python classes based on the names
    of the directories found within it.

    Args:
        target_path (str): The absolute or relative path to the directory to scan.

    Returns:
        dict: A dictionary containing the names and class objects that were created.
              Returns an empty dictionary if the path does not exist or contains no directories.
    """
    print(f"Attempting to scan for directories in: '{target_path}'")

    # --- 1. Validate the path ---
    if not os.path.isdir(target_path):
        print(f"\nError: The specified path does not exist or is not a directory.")
        print("Please make sure the path is correct and accessible.")
        return {}

    created_classes = {}

    # --- 2. Find all sub-directories ---
    try:
        # Get a list of all entries in the target_path and filter for directories
        dir_names = [name for name in os.listdir(target_path)
                     if os.path.isdir(os.path.join(target_path, name))]
    except OSError as e:
        print(f"\nError: Could not access the path. Reason: {e}")
        return {}

    if not dir_names:
        print("\nNo sub-directories were found in the specified path.")
        return {}

    print(f"\nFound {len(dir_names)} directories. Creating corresponding classes...")

    # --- 3. Create a class for each directory ---
    for dir_name in dir_names:
        # Sanitize the directory name to make it a valid Python class name
        # (e.g., "diabetic retinopathy" -> "Diabetic_retinopathy")
        class_name = re.sub(r'[^0-9a-zA-Z_]', '_', dir_name).capitalize()

        # If the first character is not a letter, prepend 'C' for 'Class'
        if not class_name[0].isalpha():
            class_name = 'C' + class_name

        # Dynamically create the class using the type() function
        # Format: type(ClassName, (BaseClasses,), {attributes_and_methods})
        new_class = type(class_name, (object,), {
            '__doc__': f'Dynamically generated class from the "{dir_name}" directory.',
            'source_directory': dir_name,
            'file_count': len(os.listdir(os.path.join(target_path, dir_name)))
        })

        # Add the new class to the global scope of this script, making it accessible
        globals()[class_name] = new_class
        created_classes[class_name] = new_class
        print(f"- Created class: {class_name}")

    return created_classes

# --- Main execution block ---
if __name__ == "__main__":
    # The path to the dataset folder provided in the request.
    # IMPORTANT: Replace this with the actual path on your system if it's different.
    dataset_folder_path = "/Users/aresbandebo/PycharmProjects/Eye_CNN_Testing/Dataset_Eye_Diseases_Classification/"

    # Generate the classes
    dynamic_classes = create_classes_from_dirs(dataset_folder_path)

    # --- Verification Step ---
    # This section demonstrates how to use the dynamically created classes.
    if dynamic_classes:
        print("\n--- Verification ---")
        print("The classes have been created and are now available for use.")
        print("Let's inspect the first class that was created:\n")

        # Get the name of the first class from the dictionary
        first_class_name = list(dynamic_classes.keys())[0]

        # Access the class from the global scope using its name
        FirstClass = globals()[first_class_name]

        # Create an instance of the class
        instance = FirstClass()

        # Print some information about the instance and its class
        print(f"Class Name: {first_class_name}")
        print(f"Instance created: {instance}")
        print(f"Instance type: {type(instance)}")
        print(f"Class docstring: {instance.__doc__}")
        print(f"Original directory name: '{instance.source_directory}'")
        print(f"Number of files in directory: {instance.file_count}")


Attempting to scan for directories in: '/Users/aresbandebo/PycharmProjects/Eye_CNN_Testing/Dataset_Eye_Diseases_Classification/'

Found 5 directories. Creating corresponding classes...
- Created class: Cataract
- Created class: Uveitis
- Created class: Conjunctivitis
- Created class: Eyelid
- Created class: Normal

--- Verification ---
The classes have been created and are now available for use.
Let's inspect the first class that was created:

Class Name: Cataract
Instance created: <__main__.Cataract object at 0x30d878f20>
Instance type: <class '__main__.Cataract'>
Class docstring: Dynamically generated class from the "Cataract" directory.
Original directory name: 'Cataract'
Number of files in directory: 544


In [4]:
  # Create a list of the class names from the dictionary keys
if dynamic_classes:
    class_names = list(dynamic_classes.keys())
    print("\n--- List of All Class Names ---")
    print("A list called 'class_names' has been created with the names of all generated classes.")
    print(class_names)


--- List of All Class Names ---
A list called 'class_names' has been created with the names of all generated classes.
['Cataract', 'Uveitis', 'Conjunctivitis', 'Eyelid', 'Normal']


In [5]:

#
#class_names = ["Cataract","Conjunctivitis","Eyelid","Normal","Uveitis"]
#print(f"Found classes: {class_names}")
def load_data(DATA_DIR):
    """
    Loads images and their corresponding labels from a directory.

    Args:
        data_dir (str): The path to the directory containing subdirectories
                        of images, where each subdirectory name is a class label.

    Returns:
        tuple: A tuple containing two lists:
               - images (list): A list of image data as numpy arrays.
               - labels (list): A list of corresponding string labels.
    """
    images = []
    labels = []


In [6]:

    # Get the class names (folder names)

    for class_name in class_names:
        class_path = os.path.join(DATA_DIR, class_name)

        # Skip files that are not directories
        if not os.path.isdir(class_path):
            continue

        print(f"Loading images from: {class_name}")
        for image_name in os.listdir(class_path):
            image_path = os.path.join(class_path, image_name)

            try:
                # Read the image using OpenCV (which loads in BGR format)
                image_bgr = cv2.imread(image_path)

                # --- IMPORTANT COLOR CORRECTION ---
                # Matplotlib expects images in RGB format. We must convert from BGR to RGB.
                # Failure to do this will result in incorrect colors (the blueish tint).
                image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)

                # Resize the image to a consistent size
                resized_image_rgb = cv2.resize(image_rgb, (IMG_WIDTH, IMG_HEIGHT))

                # Append the correctly colored, resized image and its label
                images.append(resized_image_rgb)
                labels.append(class_name)
            except Exception as e:
                print(f"Error loading image {image_path}: {e}")

    return images, labels




Loading images from: Cataract
Error loading image /Users/aresbandebo/PycharmProjects/Eye_CNN_Testing/Dataset_Eye_Diseases_Classification/Cataract/189.jpg: name 'images' is not defined
Error loading image /Users/aresbandebo/PycharmProjects/Eye_CNN_Testing/Dataset_Eye_Diseases_Classification/Cataract/162.jpg: name 'images' is not defined
Error loading image /Users/aresbandebo/PycharmProjects/Eye_CNN_Testing/Dataset_Eye_Diseases_Classification/Cataract/176.jpg: name 'images' is not defined
Error loading image /Users/aresbandebo/PycharmProjects/Eye_CNN_Testing/Dataset_Eye_Diseases_Classification/Cataract/aug_807.jpg: name 'images' is not defined
Error loading image /Users/aresbandebo/PycharmProjects/Eye_CNN_Testing/Dataset_Eye_Diseases_Classification/Cataract/406.jpg: name 'images' is not defined
Error loading image /Users/aresbandebo/PycharmProjects/Eye_CNN_Testing/Dataset_Eye_Diseases_Classification/Cataract/412.jpg: name 'images' is not defined
Error loading image /Users/aresbandebo/Pyc

SyntaxError: 'return' outside function (3960495160.py, line 32)

In [ ]:
# --- Main script execution ---
if __name__ == "__main__":
    # 1. Validate the DATA_DIR path
    if DATA_DIR == 'path/to/your/dataset':
        print("="*50)
        print("ERROR: Please update the 'DATA_DIR' variable in the script.")
        print("You need to change the default value to the actual path of your dataset.")
        print("="*50)
    elif not os.path.exists(DATA_DIR):
        print("="*50)
        print(f"ERROR: The path specified in DATA_DIR does not exist.")
        print(f"Path provided: '{DATA_DIR}'")
        print(f"Current working directory is: '{os.getcwd()}'")
        print("Please double-check that the path is correct and the script has permission to access it.")
        print("="*50)
    elif not os.path.isdir(DATA_DIR):
        print("="*50)
        print(f"ERROR: The path specified in DATA_DIR is not a directory.")
        print(f"Path provided: '{DATA_DIR}'")
        print("Please ensure DATA_DIR points to the folder containing your class subfolders.")
        print("="*50)
    else:
        # 2. Load the images and labels
        images, labels = load_data(DATA_DIR)

        # 3. Convert lists to NumPy arrays
        images = np.array(images, dtype='float32')
        labels = np.array(labels)

        # 4. Normalize image data
        # Scale pixel values from [0, 255] to [0, 1]
        images /= 255.0

        # 5. Encode string labels to integers
        label_encoder = LabelEncoder()
        integer_labels = label_encoder.fit_transform(labels)

        # 6. One-hot encode the integer labels
        # This is necessary for categorical cross-entropy loss function
        num_classes = len(np.unique(integer_labels))
        categorical_labels = to_categorical(integer_labels, num_classes=num_classes)

        # 7. Split the data into training, validation, and testing sets
        # First, split into a temporary training set (80%) and a testing set (20%)
        (X_train_temp, X_test, y_train_temp, y_test) = train_test_split(
            images, categorical_labels, test_size=0.20, stratify=categorical_labels, random_state=42
        )

        # Now, split the temporary training set into the final training set (75% of 80% = 60%)
        # and a validation set (25% of 80% = 20%)
        (X_train, X_val, y_train, y_val) = train_test_split(
            X_train_temp, y_train_temp, test_size=0.25, stratify=y_train_temp, random_state=42
        )

        # --- Output the results ---
        print("\n--- Data Preprocessing Summary ---")
        print(f"Total images loaded: {len(images)}")
        print(f"Number of classes: {num_classes}")
        print(f"Shape of training data (images): {X_train.shape}")
        print(f"Shape of training data (labels): {y_train.shape}")
        print(f"Shape of validation data (images): {X_val.shape}")
        print(f"Shape of validation data (labels): {y_val.shape}")
        print(f"Shape of testing data (images): {X_test.shape}")
        print(f"Shape of testing data (labels): {y_test.shape}")
        print("\nOriginal Labels:", label_encoder.classes_)
        print("Encoded Labels Mapping:")
        for i, class_name in enumerate(label_encoder.classes_):
            print(f"  {class_name} -> {i}")

        print("\nThe Data is now ready for training a CNN!")

In [ ]:
# --- 8. Verify the data ---
print("\n--- Data Verification ---")
total_samples = len(images)
print(f"Training set:   {len(X_train)} samples ({len(X_train)/total_samples:.2%})")
print(f"Validation set: {len(X_val)} samples ({len(X_val)/total_samples:.2%})")
print(f"Test set:       {len(X_test)} samples ({len(X_test)/total_samples:.2%})")


In [ ]:
# Function to display sample images
def show_sample_images(sample_images, sample_labels, class_names_map):
    plt.figure(figsize=(12, 12))
    for i in range(min(16, len(sample_images))): # Show up to 16 images
        plt.subplot(4, 4, i + 1)
        plt.imshow(sample_images[i])
        # Convert one-hot encoded label back to integer, then to class name
        label_index = np.argmax(sample_labels[i])
        plt.title(class_names_map[label_index])
        plt.axis("off")
    plt.suptitle("Sample Images from Training Set", fontsize=16)
    plt.show()

# Display some images from the training set to verify
print("\nDisplaying a sample of training images...")
show_sample_images(X_train, y_train, label_encoder.classes_)

In [ ]:
# Step 4: Build the Convolutional Neural Network (CNN) Model
# The model architecture is very similar to the one we used for Fashion MNIST.
# The key difference is the `input_shape`. We now specify (32, 32, 3) to
# account for the 32x32 pixel size and 3 color channels.

model = models.Sequential()
# Layer 1: Convolutional layer. 32 filters, 3x3 kernel size.
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)))
# Layer 2: Pooling layer.
model.add(layers.MaxPooling2D((2, 2)))
# Layer 3: Another Convolutional layer. We increase filters to 64.
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
# Layer 4: Another Pooling layer.
model.add(layers.MaxPooling2D((2, 2)))
# Layer 5: A final Convolutional layer.
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

# Now, let's add the final Dense (fully connected) layers for classification.
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(num_classes)) # Output layer with 5 neurons for our 5 classes

# Let's see the model summary.
model.summary()



In [ ]:
# Step 5: Compile the Model
# We configure the model for training.

model.compile(optimizer='adam',
              # For this loss function, it's good practice to set from_logits=True
              # because our model's output layer doesn't have a softmax activation function.
              # This can sometimes be more numerically stable.
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])



In [ ]:
# Step 6: Train the Model
# We'll train for 10 epochs, just like before. Training on CIFAR-10 will take
# a bit longer than on Fashion MNIST because the images are more complex.

print("\nStarting model training...")
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_test, y_test))
print("Model training finished.")



In [ ]:
# Step 7: Evaluate the Model
# Let's check the final performance on the test set.

print("\nEvaluating model on the test dataset...")
test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=2)
print(f'\nTest accuracy: {test_acc:.4f}')
print(f'Test loss: {test_loss:.4f}')
# Note: Getting around 70% accuracy on CIFAR-10 with a simple model like this is a good result!



In [ ]:
# Step 8: Visualize Training Results
# Plotting the history helps us understand the training process.

plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label = 'Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label = 'Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend(loc='upper right')

plt.show()



In [ ]:
    # Assume probability_model, predictions, X_test, y_test are already defined.
# And assume class_names is defined, for example:
# class_names = ['Class 0', 'Class 1', 'Class 2', 'Class 3', 'Class 4', 'Class 5']

# --- CORRECTED CODE ---
probability_model = tf.keras.Sequential([model, tf.keras.layers.Softmax()])
predictions = probability_model.predict(X_test)
print(predictions)
print(y_test)
# FIX: Add class_names as a parameter to the function
def plot_image_refactored(predictions_array, true_label, img, class_names):
  # No changes needed here if you were using np.argmax(true_label)
  # If true_label is just an integer, use int(true_label)
  try:
    true_label = np.argmax(true_label)
  except (ValueError, TypeError):
    true_label = int(true_label)

  plt.grid(False)
  plt.xticks([])
  plt.yticks([])
  plt.imshow(img, cmap=plt.cm.binary)

  predicted_label = np.argmax(predictions_array)
  color = 'blue' if predicted_label == true_label else 'red'

  plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)

# FIX: Add class_names as a parameter here as well
def plot_value_array_refactored(predictions_array, true_label, class_names):
  num_classes = len(class_names)

  try:
    true_label = np.argmax(true_label)
  except (ValueError, TypeError):
    true_label = int(true_label)

  plt.grid(False)
  plt.xticks(range(num_classes), class_names, rotation=90)
  plt.yticks([])
  thisplot = plt.bar(range(num_classes), predictions_array, color="#777777")
  plt.ylim([0, 1])

  predicted_label = np.argmax(predictions_array)
  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('blue')

# The plotting loop
num_rows = 5
num_cols = 3
num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for i in range(num_images):
  plt.subplot(num_rows, 2*num_cols, 2*i+1)
  # FIX: Pass class_names when you call the function
  plot_image_refactored(predictions[i], y_test[i], X_test[i], class_names)
  plt.subplot(num_rows, 2*num_cols, 2*i+2)
  print(class_names)
  #plot_value_array_refactored(predictions[i], y_test[i], class_names)
  # FIX: Pass class_names here too
  plot_value_array_refactored(predictions[i], y_test[i], class_names)

plt.tight_layout()
plt.show()